# Python-equivalent commands for bumphunting

In [1]:
#import csv ##module required so far just to read CSV data files output by minfi processing in R
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [3]:
##Read in the minfi processed output matrices needed to run bumphunter code
betaVals = pd.read_csv("Sample_Beta_value_matrix_quantilePreprocessed.csv",index_col=0)
phenoDesign = pd.read_csv("Sample_phenotype_design_matrix.csv")
probePositions = pd.read_csv("Sample_probe_genomic_positions.csv",index_col=0)

In [4]:
probePositions.head()

,Chromosome,Position
cg13869341,chr1,15865
cg14008030,chr1,18827
cg12045430,chr1,29407
cg20826792,chr1,29425
cg00381604,chr1,29435


In [5]:
betaVals.head()

,GSM1711360_9247377093_R02C01,GSM1711361_9247377093_R03C01,GSM1711362_9247377093_R06C02,GSM1711363_9247377085_R04C02,GSM1711364_9247377093_R05C01,GSM1711365_9247377093_R02C02
cg13869341,0.853966,0.859463,0.841423,0.864711,0.866001,0.889798
cg14008030,0.804748,0.808577,0.782708,0.796127,0.801304,0.858243
cg12045430,0.176411,0.160218,0.145978,0.165587,0.200644,0.195475
cg20826792,0.476619,0.443585,0.342943,0.521183,0.564243,0.540803
cg00381604,0.232571,0.207345,0.167195,0.170035,0.191317,0.162296


In [6]:
betaVals.shape

(485512, 6)

The first set of commands within the <span style="font-family:Courier">bumphunterEngine</span> check which arguments were specified when the <span style="font-family:Courier">bumphunter</span> function was called, check if they're in the correct format, and produce informative warnings if any input is incorrect.

Then the <span style="font-family:Courier">bumphunterEngine</span> calls several functions from R's <span style="font-family:Courier">foreach</span> package to see if multiple cores were registered before <span style="font-family:Courier">bumphunter</span> was called. This sets up the makes it possible for the more time-consuming <span style="font-family:Courier">bumphunterEngine</span> functions to process in parallel as coded.

<span style="color:red"><b>Parallel processing has NOT been addressed yet in this python implementation</b></span>

The first backend function <span style="font-family:Courier">bumphunterEngine</span> calls is <span style="font-family:Courier">clusterMaker</span>, which assigns genomic locations to sets of clusters if they are within <span style="font-family:Courier">maxGap</span> of one another (if clusters were not already pre-assigned in one of the function arguments).

In [17]:
##if (is.null(cluster))
    ##cluster <- clusterMaker(chr, pos, maxGap = maxGap)

## clusterMaker() commands

In [29]:
##clusterMaker function args (lists a different default maxGap than bumphunter())
    ##clusterMaker <- function(chr, pos, assumeSorted = FALSE, maxGap=300)
    
#nonaIndex <- which(!is.na(chr) & !is.na(pos))   ##integer vector of probe indices from 1-485512
#Indexes <- split(nonaIndex, chr[nonaIndex])  ##24 integer vectors (one for each chromosome)
#clusterIDs <- rep(NA, length(chr))
#LAST <- 0

##Set maximum distance in bp allowed between probes in the same cluster
maxGap = 300
assumeSorted = False   ##don't assume the genomic positions are listed in order of chromosomal location

##Drop NA values from the dataframe of chromosome IDs and genomic locations for each probe
if probePositions.isnull().values.any():
    probePositions.dropna()

##set "last" counter to zero (keeps count of how many clusters were assigned on previous chromosomes in following loop)
last = 0

##Create list of all chromosome IDs present in dataset
probeChr = set(probePositions["Chromosome"])
probeChr = list(probeChr)
probeChr[:5]

##list to hold cluster ID number for each genomic position
clusterIDs = []

R code (commented out above Python commands) splits the probe positions by chromosome into a list of vectors, but is that really necessary? I'm going to keep the pandas dataframe together and select by chromosome when it comes to it. Might change this if that's unwieldy or slow.

In [30]:
#assumeSorted = FALSE
#for(i in seq(along = Indexes)){  ##loops 24 times, once for each chromosome
    #Index <- Indexes[[i]]
    #x <- pos[Index]   ##select probe positions for that chromosome
    #if(!assumeSorted){   ##make sure probe genomic locations are in order
        #Index <- Index[order(x)]
        #x <- pos[Index]
    #}
    ##diff(x) to calculate distance between each sequential position
    #y <- as.numeric(diff(x) > maxGap)  ##binary vector where 0 = distance < maxGap and 1 = distance < maxGap
    #z <- cumsum(c(1, y))  ##cumulative sum of this binary vector
    #clusterIDs[Index] <- z + LAST  ##turn this cumulative sum vector into the cluster ID numbers
    #LAST <- max(z) + LAST  ##record the last clusterID to start at the next number for the following chromosome
#}
#head(clusterIDs)


##Loop 24 times (once for each chromosome since clusters can't span across chromosomes)
for i in probeChr:
    ##Select just the rows of the dataframe corresponding to that chromosome
    chr_rows = probePositions[probePositions.Chromosome==i]
    ##Create a series of the numeric genomic locations for each probe on that chromosome
    x = chr_rows.Position
    ##Make sure they're sorted in ascending order
    if not assumeSorted:
        x = x.sort_values()
    ##Calculate the difference between each successive position
    diff_x = np.diff(x)
    ##Create a list of 0s (difference is <maxGap) and 1s (difference is >maxGap)
    y = (diff_x > maxGap)*1
    ##diff is always one item shorter than the series from which it was calculated, so add one 1 to the start of y 
    #y.insert(0,1)
    y_all = np.insert(y,0,1)
    ##Take the cumulative sum of these zeroes and ones to assign cluster ID numbers
        ##that start a new cluster each time the probes are >maxGap apart
    z = np.cumsum(y_all)
    ##Add the final cluster number from the previous chromosome to z so that this chromosome starts at the next ID #
    #z_plus = [1+i for i in z]
    z_plus = z+last
    ##Concatenate this chromosome's list of cluster IDs with the running list of cluster ID numbers
    clusterIDs = np.append(clusterIDs,z_plus)
    ##Adjust the "last" counter to the final cluster ID number for this chromosome
    last = max(z_plus)
    

In [8]:
len(clusterIDs)
clusterIDs[-10:]

array([242697., 242697., 242697., 242698., 242699., 242700., 242700.,
       242701., 242702., 242703.])

After designating cluster ID numbers, <span style="font-family:Courier">bumphunterEngine</span> makes decisions about smoothing and weighting the data depending on the inputs the user provided for the function. Whether the data is eventually weighted or not, it is first processed through the <span style="font-family:Courier">.getEstimate</span> method.

In [ ]:
##if (useWeights & smooth) { 
    ##tmp <- .getEstimate(mat = mat, design = design,
                        ##coef = coef, full = TRUE)
    ##rawBeta <- tmp$coef
    ##weights <- 1/tmp$sigma
    ##rm(tmp)
##} else {
    ##rawBeta <- .getEstimate(mat = mat, design = design, coef = coef,
            ##full = FALSE)
    ##weights <- NULL
##}

## .getEstimate() commands

<span style="color:red"><b>Currently doesn't handle a user input matrix for the <span style="font-family:Courier">permutations</span> argument (assumes argument has been left blank and defaults to None).</b></span>

Found issues when trying to find corresponding <span style="font-family:Courier">numpy</span> functions for the <span style="font-family:Courier">R</span> matrix operations used in solving the linear regression. The following is a cheatsheet I came up with after experimenting with simple test matrices to see what commands produced the same output in both languages:

<b>R</b> | <b>Python</b>
 --- | --- 
A %*% B | np.dot(A,B)
t(A) | A.T
crossprod(A,B) | np.dot(A.T,B)
tcrossprod(A,B) | np.dot(A,B.T)

In [9]:
##.getEstimate <- function(mat, design, coef, B=NULL, permutations=NULL,
                         ##full=FALSE) {
    ##v <- design[,coef]
    ##A <- design[,-coef, drop=FALSE]
    ##qa <- qr(A)
    ##S <- diag(nrow(A)) - tcrossprod(qr.Q(qa))

    ##vv <- if (is.null(B)) matrix(v, ncol=1) else{
        ##if(is.null(permutations)){
            ##replicate(B, sample(v))
        ##} else{
            ##apply(permutations,2,function(i) v[i])
        ##}
    ##}
            
            
    ##sv <- S %*% vv
    ##vsv <- diag(crossprod(vv,sv))
    
    ##b <- (mat %*% crossprod(S, vv)) / vsv
    ##if(!is.matrix(b))
        ##b <- matrix(b, ncol = 1)
    ##if (full) {
        ##sy <- mat %*% S
        ##df.residual <- ncol(mat) - qa$rank - 1
        ##if (is.null(B)) {
            ##sigma <- matrix(sqrt(rowSums((sy - tcrossprod(b, sv))^2) / df.residual), ncol=1)
        ##} else {
            ##sigma <- b
            ##tmp <- sy
            ##for (j in 1:B) {
                ##tmp <- tcrossprod(b[,j], sv[,j])
                ##sigma[,j] <- rowSums((sy-tmp)^2)
            ##}
            ##sigma <- sqrt(sigma/df.residual)
        ##}
        ##out <- list(coef=b,
                    ##sigma=sigma,
                    ##stdev.unscaled=sqrt(1/vsv),
                    ##df.residual=df.residual)
        ##if (is.null(B)) out$stdev <- as.numeric(out$stdev)
    ##} else {
        ##out <- b
    ##}
    ##return(out)
##}


##Default values
##coef = 2 --> no need for this in Python implementation
B=None     ##Works with B > 0 when tested
permutations=None
full=False    ##Works with full = True when tested

##Select the columns of the data frame that are actual phenotype descriptors
v = phenoDesign.loc[:,phenoDesign.columns!="(Intercept)"]
##Select the first, non-phenotype column of the design data frame
Adf  = phenoDesign["(Intercept)"]
##Save this one dataframe column as a numpy array
A = np.array(Adf)
##Make sure the array is 2 dimensional
A = A.reshape(-1,1)
##Calculate the QR decomposition of the matrix
    ####May want to replace with scipy.linalg.qr to get rank more legitimately later
qa = np.linalg.qr(A)
##Create a diagonal matrix with the same dimensions as the number of samples
diagonal = np.zeros((A.shape[0], A.shape[0]), int)
np.fill_diagonal(diagonal, 1)
##Subtract the cross product of qa times the transposition of itself from the diagonal matrix
qa_mat = np.matrix(qa[0])
S = diagonal - np.dot(qa_mat,qa_mat.T)

##If the user didn't specify a number of permutations in the function input
if B is None:
    ##use the array version of v (the phenotype column(s)) for vv
    vv = np.array(v)
else:
    ##If the user didn't provide a matrix for the permutations argument
    if permutations is None:
        ##convert pandas series v into an array and flatten it to one dimensional
        v_arr = np.array(v)
        v_arr = v_arr.reshape(v_arr.shape[0])
        ##Randomly permute the phenotypes among the samples B times
        vv = np.random.choice(v_arr,v_arr.shape[0],replace=False)
        for i in range(B-1):
            next_sample = np.random.choice(v_arr,v_arr.shape[0],replace=False)
            vv = np.vstack((vv,next_sample))
        vv = vv.T   ##Phenotype permutation matrix


vv_mat = np.matrix(vv)
S_mat = np.matrix(S)
sv = np.dot(S_mat,vv_mat)
vsv = np.diag(np.dot(vv_mat.T,sv))
##Bring the matrix of methylation values for each probe into the math
beta_mat = np.matrix(betaVals)
b = np.dot(beta_mat,np.dot(S_mat.T, vv_mat)) / vsv

##If b doesn't turn out to be a matrix, shape the values into a one-column matrix
if type(b) is not np.matrix:
    b = np.matrix(b)
    b = b.reshape(b.shape[0]*b.shape[1],1)
if full:
    sy = np.dot(beta_mat,S_mat)
    ##Calculate degrees of freedom?
    df_residual = beta_mat.shape[1] - qa_mat.shape[1] -1
    ##If no number of permutations was specified by the user
    if B is None:
        sigma = np.matrix(np.sqrt(np.sum(np.square(sy-np.dot(b,sv.T)),axis=1)/df_residual))
        sigma = sigma.reshape(sigma.shape[0]*sigma.shape[1],1)
    else:
        sigma = b
        tmp = sy
        for j in range(B):
            tmp = np.dot(b[:,j],sv[:,j].T)
            sigma[:,j] = np.sum(np.square(sy-tmp),axis=1)
        sigma = np.sqrt(sigma/df_residual)
    coef = b
    stdev_unscaled = np.sqrt(1/vsv)
        
##Below are the data structures returned by .getEstimate()

        ##out <- list(coef=b,
                    ##sigma=sigma,
                    ##stdev.unscaled=sqrt(1/vsv),
                    ##df.residual=df.residual)
        ##if (is.null(B)) out$stdev <- as.numeric(out$stdev)
    ##} else {
        ##out <- b
    ##}
    ##return(out)
##}

        

## Proceeding without weights but with smoothing

The output of the previous <span style="font-family:Courier">.getEstimate</span> commands are used as the input <span style="font-family:Courier">rawBeta</span> for the smoothing function.

In [ ]:
    ##if (smooth) {
        ##if (verbose)
            ##message("[bumphunterEngine] Smoothing coefficients.")
        ##beta <- smoother(y = rawBeta, x = pos, cluster = cluster,
            ##weights = weights, smoothFunction = smoothFunction,
            ##verbose = subverbose, ...)
        ##Index <- which(beta$smoothed)
        ##beta <- beta$fitted
    ##} else {
        ##beta <- rawBeta
        ##Index <- seq(along = beta)
    ##}

In [ ]:
##smoother <- function(y, x=NULL, cluster, weights=NULL,
                     ##smoothFunction, verbose=TRUE, ...) {

## y = rawBeta --> b output from previous commands
## x = pos --> probePositions.Position
## cluster --> clusterIDs

## smoother() function commands

<span style="color:red"><b>This function <i>can</i> use parallel processing, but it has NOT been implemented yet in this python translation</b></span>

In [10]:
##smoother <- function(y, x=NULL, cluster, weights=NULL,
                     ##smoothFunction, verbose=TRUE, ...) {
    
weights = None
verbose = True
    ##if(is.null(dim(y)))
        ##y <- matrix(y, ncol=1) ##need to change this to be more robust
if type(b) is np.matrix:   ##If b is not a matrix
    y = b
else:    ##reshape its values into a 1 column matrix
    y = np.matrix(b)
    y = y.reshape(y.shape[0]*y.shape[1],1)
    ##if(!is.null(weights) && is.null(dim(weights)))
        ##weights <- matrix(weights, ncol=1)
if (weights is not None) and (type(weights) is not np.matrix):
    weights = np.matrix(weights)
    weights = weights.reshape(weights.shape[0]*weights.shape[1],1)
        
    ##if (!getDoParRegistered())
        ##registerDoSEQ()
    ##cores <- getDoParWorkers()
cores = 1 
    
    ##Indexes <- split(seq(along=cluster), cluster)
    ##baseSize <- length(Indexes) %/% cores
    ##remain <- length(Indexes) %% cores
    ##done <- 0L
    ##IndexesChunks <- vector("list", length = cores)
Indexes = []
ID_num = clusterIDs[0]
thisInd = [1]
for i in range(1,len(clusterIDs)):
    if clusterIDs[i] == ID_num:
        thisInd.append(i)
        ID_num = clusterIDs[i]
    else:
        Indexes.append(thisInd) 
        thisInd = [i]
        ID_num = clusterIDs[i]
baseSize = int(len(Indexes)/cores)  ##number of clusters to assign to each core
remain = len(Indexes) % cores  ##leftover clusters if they didn't divide evenly among cores
done = 0   ##counter to keep track of clusters that have already been assigned to a core
IndexesChunks = [0]*cores  ##list to hold each cores' list of clusters
    
    
    ##for(ii in 1:cores) {
        ##if(remain > 0) {   ##
            ##IndexesChunks[[ii]] <- done + 1:(baseSize + 1)
            ##remain <- remain - 1L
            ##done <- done + baseSize + 1L
        ##} else {
            ##IndexesChunks[[ii]] <- done + 1:baseSize
            ##done <- done + baseSize
        ##}
    ##}
    ##For each available core, create a list of 
for i in range(cores):
    if remain > 0:
        baseList = range(baseSize+1)
        IndexesChunks[i] = [num+done for num in baseList]
        remain -= 1
        done += baseSize + 1
    else:
        baseList = range(baseSize)
        IndexesChunks[i] = [num+done for num in baseList]
        done += baseSize
    
    ##Commands for parallelization with R's foreach
    
    ##IndexesChunks <- lapply(IndexesChunks, function(idxes) {
        ##do.call(c, unname(Indexes[idxes]))
    ##})
    ##idx <- NULL ## for R CMD check
    ##ret <- foreach(idx = iter(IndexesChunks), .packages = "bumphunter") %dorng% {
    
    ####APPLY SMOOTHING FUNCTION#####
    
        ##sm <- smoothFunction(y=y[idx,], x=x[idx], cluster=cluster[idx],
                             ##weights=weights[idx,], verbose=verbose, ...)
        ##c(sm, list(idx = idx))
    ##}
    
##smoothFunction can be one of three options as specified in the smoother() arguments:
    ##locfit
    ##loess
    ##runmed
    
if smoothFunction == "loess":
    sm.nonparametric.lowess(y,x)
    
    
elif smoothFunction == "runmed":
    pd.rolling_median(y,window)
    ##window in this case is an integer --> need to update for each cluster somehow
    
    ##attributes(ret)[["rng"]] <- NULL
    ## Paste together results from different workers
    ##ret <- reduceIt(ret)

    ## Now fixing order issue
    ##revOrder <- ret$idx
    ##names(revOrder) <- seq_along(ret$idx)
    ##revOrder <- sort(revOrder)
    ##revOrder <- as.integer(names(revOrder))
    
    ##ret$smoother <- ret$smoother[1]
    ##ret$fitted <- ret$fitted[revOrder,,drop=FALSE]
    ##ret$smoothed <- ret$smoothed[revOrder]
    ##ret$idx <- NULL
    ##return(ret)  
##}

In [11]:
baseSize

242702

### Python equivalents for R's smoothing function options used in bumphunter

<span style="font-family:Courier">[locfit](http://ugrad.stat.ubc.ca/R/library/locfit/html/locfit.html)</span> - returns a fit object fitted to a local regression model (methods are outlined in the book [Local Regression and Likelihood](http://web.ipac.caltech.edu/staff/fmasci/home/astro_refs/LocalRegressionBook_1999.pdf))  
<b>Python options:</b>  
* [Python wrapper for Locfit C routines](https://github.com/chairmank/python-locfit) that uses <span style="font-family:Courier">numpy</span> conventions in a github repository. Hasn't been updated in years, but says it has an MIT Open Source Software license (link is dead).
* Github project for a [Gradient Boosting Machine that uses Locfit](https://github.com/materialsproject/gbml/) for local regression. It's been updated recently, but hard to tell how you actually use it in Python... Seems to rely on wrapper project above!

<span style="font-family:Courier">[loess](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/loess.html)</span> - also uses local regression to smooth polynomial  
<b>Python options:</b>  
* ~~An individual wrote up [code for a LOESS non-parametric smoothing function](http://www.jtrive.com/loess-nonparametric-scatterplot-smoothing-in-python.html) (dependencies: <span style="font-family:Courier">pandas, numpy, scipy</span>)~~
* A function for lowess smoothing is available [in the <span style="font-family:Courier">statsmodel</span> module](https://www.statsmodels.org/dev/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html)
* ~~The <span style="font-family:Courier">scikit-misc</span> module also offers a [loess function](https://has2k1.github.io/scikit-misc/loess.html).~~
* ~~Another individual on github made a [lowess smoothing script](https://gist.github.com/agramfort/850437). (dependencies: <span style="font-family:Courier">numpy, scipy</span>)~~

<span style="font-family:Courier">[runmed](https://www.rdocumentation.org/packages/stats/versions/3.5.1/topics/runmed)</span> - calculates a running median across a list of values for a given window  
<b>Python options:</b>
* <span style="font-family:Courier">pandas</span> has a [rolling median function](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.rolling_median.html), sold!


## smoothFunction() commands --> loessByCluster() smoothing

In the middle of <span style="font-family:Courier">smoother</span>, the R function calls one of three different <span style="font-family:Courier">smootherFunction</span>s depending on user specification. The default is <span style="font-family:Courier">locfit</span>, but another option is <span style="font-family:Courier">loess</span> which is also based on local regression.

In [ ]:
##sm <- smoothFunction(y=y[idx,], x=x[idx], cluster=cluster[idx],
                             ##weights=weights[idx,], verbose=verbose, ...)

##loessByCluster <- function(y, x=NULL, cluster, weights= NULL,
                         ##bpSpan = 1000, minNum=7, minInSpan=5,
                         ##maxSpan=1, verbose=TRUE) 
        
        
# x = chromosome positions
# y = regression beta values

In [1]:
    ## Depends on limma
    ## bpSpan is in basepairs
    ## assumed x are ordered
    ## if y is vector change to matrix
    
x = probePositions.Position
y = b
bpSpan = 1000
minNum = 7   ##minimum number of probes in a cluster to be able to smooth the cluster
minInSpan = 5
maxSpan = 1
verbose = True
    
    ##if(is.null(dim(y)))
        ##y <- matrix(y, ncol=1) ##need to change this to be more robust
    ##if(!is.null(weights) && is.null(dim(weights)))
        ##weights <- matrix(weights,ncol=1)
    
    ##if(is.null(x))
        ##x <- seq(along=y)
if x is None:   ##if no x variable is provided, use a sequential list the length of y
    x = range(y.shape[0])
        
    ##if(is.null(weights))
        ##weights <- matrix(1, nrow=nrow(y), ncol=ncol(y))
if weights is None:  ##if no weights are provided, make a matrix the same size as y
    weights = np.zeros((y.shape[0],y.shape[1]))
    weights += 1   ##with every weight equal to 1
        
    ##Indexes <- split(seq(along=cluster), cluster) --> done in cell above for smoother()
    ##clusterL <- sapply(Indexes, length)
    ##spans <- rep(NA, length(Indexes))
    ##smoothed <- rep(TRUE, nrow(y))
clusterL = []    ##list that counts the number of probes in each cluster
for i in range(len(Indexes)):
    clusterL.append(len(Indexes[i]))
spans = [None]*len(Indexes)   
smoothed = [True]*y.shape[0]
    
    ##for(i in seq(along=Indexes)) {
for i in range(len(Indexes)):
        ##if(verbose) if(i %% 1e4 == 0) cat(".")
        ##Index <- Indexes[[i]]
    Index = Indexes[i]
        ##if(clusterL[i] >= minNum) {
    ##if this cluster contains the minimum number of probes required for smoothing
    if (clusterL[i] >= minNum)   
            ##span = bpSpan/median(diff(x[Index]))/length(Index) # make a span
            ##if(span > maxSpan) span <- maxSpan
            ##spans[i] <- span
        span = bpSpan/np.median(np.diff(x[Index]))/len(Index)
        if span > maxSpan:
            span = maxSpan
        spans[i] = span
            ##if(span*length(Index)>minInSpan){
                ##this can be parallelized
                ##for(j in 1:ncol(y)){
                    ##y[Index,j] <- limma::loessFit(y[Index,j], x[Index], span = span,
                              ##weights = weights[Index,j])$fitted
                ##}
        if span*len(Index) > minInSpan:
            for j in range(y.shape[0]):
                y[Index,j]
            ##} else{
                ##y[Index,] <- NA
                ##smoothed[Index] <- FALSE
            ##}
        ##} else{
            ##y[Index,] <- NA
            ##spans[i] <- NA
            ##smoothed[Index] <- FALSE
        ##}
    ##}
    
    ##return(list(fitted=y, smoothed=smoothed, smoother="loess"))
##}


SyntaxError: invalid syntax (<ipython-input-1-070dc9aefd46>, line 3)

In [47]:
x = probePositions.Position
            ##span = bpSpan/median(diff(x[Index]))/length(Index) # make a span
            ##if(span > maxSpan) span <- maxSpan
            ##spans[i] <- span
Index = Indexes[16]
bpSpan=1000
bpSpan/np.median(np.diff(x[Index]))/len(Index)
coef.shape

NameError: name 'coef' is not defined

## smoothFunction() commands --> runmed smoothing

In [31]:
clusterIDs[:50]

array([ 1.,  1.,  1.,  2.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.,  9.,
        9.,  9.,  9., 10., 11., 11., 11., 12., 13., 14., 15., 16., 17.,
       17., 17., 17., 17., 17., 18., 19., 19., 20., 21., 21., 22., 23.,
       24., 25., 26., 27., 27., 27., 27., 27., 28., 28., 28.])

In [13]:
weights

NameError: name 'weights' is not defined